In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# %%
# Install required packages
!pip install youtube-transcript-api transformers torch flask flask-cors pyngrok huggingface_hub accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.0/485.0 kB 9.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninst

In [3]:
# %%
import os
import json
from urllib.parse import urlparse, parse_qs
from youtube_transcript_api import YouTubeTranscriptApi
from transformers import pipeline
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import threading
import time
from huggingface_hub import login


2025-09-30 08:52:02.413864: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759222322.747437      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759222322.840319      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# %%
def extract_video_id(url: str) -> str:
    """
    Extract the YouTube video ID from a URL.
    Supports both youtube.com and youtu.be formats.
    """
    try:
        parsed = urlparse(url)
        
        # Handle youtu.be format
        if 'youtu.be' in parsed.netloc:
            return parsed.path[1:]  # Remove the leading '/'
        
        # Handle youtube.com format
        qs = parse_qs(parsed.query)
        video_ids = qs.get('v')
        
        if not video_ids:
            raise ValueError(f"No video id found in URL: {url}")
        return video_ids[0]
    except Exception as e:
        raise ValueError(f"Invalid YouTube URL: {str(e)}")

def chunk_text(text, chunk_size=500):
    """
    Split text into chunks for summarization.
    """
    words = text.split()
    chunks = []
    
    for i in range(0, len(words), chunk_size):
        chunk = " ".join(words[i:i+chunk_size])
        chunks.append(chunk)
    
    return chunks

print("Helper functions loaded successfully!")


Helper functions loaded successfully!


In [5]:
# %%
# Login to Hugging Face (optional, for private models)
try:
    login()
    print("Logged in to Hugging Face successfully!")
except:
    print("Hugging Face login skipped - using public models only")

# Initialize the summarization model
print("Loading summarization model...")
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
print("Model loaded successfully!")


Logged in to Hugging Face successfully!
Loading summarization model...


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


Model loaded successfully!


In [6]:
# %%
# Create Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

@app.route('/health', methods=['GET'])
def health_check():
    """Health check endpoint"""
    return jsonify({"status": "healthy", "message": "YouTube Summarizer API is running!"})

@app.route('/summarize', methods=['POST'])
def summarize_video():
    """
    Summarize YouTube video endpoint
    Expects JSON: {"url": "youtube_video_url"}
    Returns JSON: {"summary": "video_summary"} or {"error": "error_message"}
    """
    try:
        # Get video URL from request
        data = request.get_json()
        if not data or 'url' not in data:
            return jsonify({"error": "Missing 'url' parameter in request body"}), 400
        
        video_url = data['url'].strip()
        if not video_url:
            return jsonify({"error": "Empty URL provided"}), 400
        
        print(f"Processing video URL: {video_url}")
        
        # Extract video ID
        try:
            video_id = extract_video_id(video_url)
            print(f"Extracted video ID: {video_id}")
        except ValueError as e:
            return jsonify({"error": str(e)}), 400
        
        # Get transcript
        try:
            api = YouTubeTranscriptApi()
            fetched = api.fetch(video_id, languages=['en'])
            transcript_text = "\n".join(snippet.text for snippet in fetched)
            print(f"Retrieved transcript with {len(transcript_text)} characters")
            
            if len(transcript_text.strip()) == 0:
                return jsonify({"error": "No transcript available for this video"}), 400
                
        except Exception as e:
            return jsonify({"error": f"Failed to fetch transcript: {str(e)}"}), 400
        
        # Process text in chunks
        try:
            chunks = chunk_text(transcript_text, chunk_size=500)
            print(f"Split transcript into {len(chunks)} chunks")
            
            summaries = []
            for i, chunk in enumerate(chunks):
                if len(chunk.strip()) > 50:  # Only summarize meaningful chunks
                    summary = summarizer(chunk, max_length=120, min_length=30, do_sample=False)
                    summaries.append(summary[0]['summary_text'])
                    print(f"Processed chunk {i+1}/{len(chunks)}")
            
            # Combine all summaries
            final_summary = " ".join(summaries)
            print(f"Generated final summary with {len(final_summary)} characters")
            
            return jsonify({
                "summary": final_summary,
                "chunks_processed": len(summaries),
                "original_length": len(transcript_text)
            })
            
        except Exception as e:
            return jsonify({"error": f"Failed to generate summary: {str(e)}"}), 500
            
    except Exception as e:
        print(f"Unexpected error: {str(e)}")
        return jsonify({"error": f"Internal server error: {str(e)}"}), 500

print("Flask app configured successfully!")


Flask app configured successfully!


In [7]:
# Install
!pip -q install pyngrok==7.* flask uvicorn

import os
from pyngrok import ngrok, conf

# Read token from Kaggle Secrets / env
NGROK_TOKEN = "32JYankevjVACV9Gy2F0KQ2YhJ8_6Fne9bccQciyPpEtcJAKR"  # set this in Kaggle "Secrets"
assert NGROK_TOKEN, "Missing NGROK_AUTHTOKEN. Add it in Add-ons → Secrets."

# Configure ngrok
conf.get_default().auth_token = NGROK_TOKEN
# Optional: choose region ('us','eu','ap','au','sa','jp','in')
conf.get_default().region = "eu"  # change if you want
print("ngrok configured.")


ngrok configured.


In [8]:
# %%
# Set up ngrok for external access
def start_ngrok():
    """Start ngrok tunnel"""
    try:
        # Kill any existing ngrok processes
        ngrok.kill()
        
        # Start ngrok tunnel on port 5000
        public_url = ngrok.connect(5000)
        print(f"\n🌐 Ngrok tunnel established!")
        print(f"📡 Public URL: {public_url}")
        print(f"🔗 Use this URL in your Flutter app: {public_url}")
        print(f"📋 API Endpoints:")
        print(f"   - Health Check: {public_url}/health")
        print(f"   - Summarize: {public_url}/summarize")
        print(f"\n⚠️  Remember to update the baseUrl in your Flutter app!")
        
        return public_url
    except Exception as e:
        print(f"Failed to start ngrok: {str(e)}")
        return None

# Start ngrok
public_url = start_ngrok()


                                                                                                    
🌐 Ngrok tunnel established!
📡 Public URL: NgrokTunnel: "https://34ed97257a0b.ngrok-free.app" -> "http://localhost:5000"
🔗 Use this URL in your Flutter app: NgrokTunnel: "https://34ed97257a0b.ngrok-free.app" -> "http://localhost:5000"
📋 API Endpoints:
   - Health Check: NgrokTunnel: "https://34ed97257a0b.ngrok-free.app" -> "http://localhost:5000"/health
   - Summarize: NgrokTunnel: "https://34ed97257a0b.ngrok-free.app" -> "http://localhost:5000"/summarize

⚠️  Remember to update the baseUrl in your Flutter app!


In [9]:
# %%
def run_flask():
    """Run Flask app"""
    app.run(host='0.0.0.0', port=5000, debug=False)

# Start Flask server in a separate thread
print("🚀 Starting Flask server...")
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

# Wait a moment for server to start
time.sleep(2)
print("✅ Server started successfully!")
print(f"🌍 Local server: http://localhost:5000")
if public_url:
    print(f"🌐 Public URL: {public_url}")
print("\n📝 Server is ready to receive requests!")


🚀 Starting Flask server...
 * Serving Flask app '__main__'
 * Debug mode: off
✅ Server started successfully!
🌍 Local server: http://localhost:5000
🌐 Public URL: NgrokTunnel: "https://34ed97257a0b.ngrok-free.app" -> "http://localhost:5000"

📝 Server is ready to receive requests!


In [10]:
# %%
import requests

# Test the API
def test_api():
    """Test the API with a sample video"""
    test_url = "https://www.youtube.com/watch?v=E0Hmnixke2g"
    
    if public_url:
        # Test health endpoint
        try:
            health_response = requests.get(f"{public_url}/health")
            print("Health check:", health_response.json())
        except Exception as e:
            print(f"Health check failed: {e}")
        
        # Test summarization endpoint
        try:
            print(f"\n🧪 Testing with video: {test_url}")
            response = requests.post(
                f"{public_url}/summarize",
                json={"url": test_url},
                timeout=300  # 5 minutes timeout
            )
            
            if response.status_code == 200:
                result = response.json()
                print("✅ Summary generated successfully!")
                print(f"📝 Summary: {result['summary'][:200]}...")
                print(f"📊 Chunks processed: {result['chunks_processed']}")
            else:
                print(f"❌ Error: {response.json()}")
                
        except Exception as e:
            print(f"API test failed: {e}")
    else:
        print("❌ No public URL available for testing")

# Run the test
test_api()


Health check failed: No connection adapters were found for 'NgrokTunnel: "https://34ed97257a0b.ngrok-free.app" -> "http://localhost:5000"/health'

🧪 Testing with video: https://www.youtube.com/watch?v=E0Hmnixke2g
API test failed: No connection adapters were found for 'NgrokTunnel: "https://34ed97257a0b.ngrok-free.app" -> "http://localhost:5000"/summarize'
Processing video URL: https://www.youtube.com/watch?v=2-DJvo4YelY
Extracted video ID: 2-DJvo4YelY
